In [7]:
from FuncionAzul import *

# Prueba Data Corta

In [5]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test2/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test2/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test2/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test2/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test2/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test2/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test2/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = True)

CPU times: user 447 µs, sys: 0 ns, total: 447 µs
Wall time: 546 µs


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/gaszsantana/fractal/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/gaszsantana/fractal/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [2]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test1/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test1/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test1/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test1/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test1/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test1/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test1/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

22/08/02 07:09:18 WARN Utils: Your hostname, Arturos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.35 instead (on interface en0)
22/08/02 07:09:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/02 07:09:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/02 07:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:33 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/02 07:09:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/02 07:09:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:09:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

...

22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 07:12:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 0

# Prueba Data Completa

In [ ]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test2/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test2/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test2/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test2/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test2/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test2/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test2/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

22/08/21 16:24:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/21 16:24:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/21 16:24:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:24:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/21 16:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:26:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/21 16:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:28:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:28:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:28:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 1

22/08/21 16:32:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:32:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:32:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:32:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:32:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:32:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 1

22/08/21 16:33:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:33:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:33:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:33:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/21 16:34:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:34:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:34:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/21 16:34:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
